In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# AGS_001 – Foundation Knowledge
# Author: Ashok Miji | Series: AGS
# Dataset Sources: GSM8K, ARC, BoolQ, TruthfulQA

!pip install datasets transformers openai -q

from datasets import load_dataset
import pandas as pd
import random

# Load GSM8K – grade school arithmetic
gsm8k = load_dataset("gsm8k", "main", split="train[:200]")
gsm_df = pd.DataFrame(gsm8k)

# Load BoolQ – factual yes/no
boolq = load_dataset("boolq", split="train[:200]")
boolq_df = pd.DataFrame(boolq)

# Load ARC – AI2 reasoning challenge
arc = load_dataset("ai2_arc", "ARC-Challenge", split="train[:200]")
arc_df = pd.DataFrame(arc)

# Load TruthfulQA – hallucination testing
truthfulqa = load_dataset("truthful_qa", "multiple_choice", split="validation[:100]")
truth_df = pd.DataFrame(truthfulqa)

# Display samples
print("\n📘 GSM8K Sample:\n", gsm_df[['question', 'answer']].sample(3))
print("\n❓ BoolQ Sample:\n", boolq_df[['question', 'answer']].sample(3))
print("\n🧠 ARC Sample:\n", arc_df[['question', 'choices']].sample(2))
print("\n🧪 TruthfulQA Sample:\n", truth_df[['question', 'mc1_targets']].sample(2))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
t

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

...

Generating validation split:   0%|          | 0/299 [00:00<?, ? examples/s]

README.md: 0.00B [00:00, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/271k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]


📘 GSM8K Sample:
                                               question  \
167  Janet filmed a new movie that is 60% longer th...   
64   James earns $20 an hour while working at his m...   
185  Caleb and his dad went fishing at the lake.  C...   

                                                answer  
167  The first movie was 2*60=<<2*60=120>>120 minut...  
64   James earns 20*.2=$<<20*.2=4>>4 less while wor...  
185  Caleb’s dad caught 2 x 3 = <<2*3=6>>6 trouts.\...  

❓ BoolQ Sample:
                                              question  answer
89            is ferguson jenkins in the hall of fame    True
76                  is it legal to carry a sgian dubh    True
59  is there an international airport in naples italy    True

🧠 ARC Sample:
                                               question  \
46        Which planet has the longest planetary year?   
197  Which diagram best shows how energy flows thro...   

                                               choices  
46   {'

In [3]:
import openai
openai.api_key = "sk-..."  # Replace securely

def ask_chatgpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

# Example:
ask_chatgpt("Solve this math question: A school has 20 students and buys 80 apples equally. How many apples per student?")

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
